# Case based on Card and Krueger (1993), which estimates the causal effect of an increase in the state minimum wage on the employment.

* On April 1, 1992, New Jersey raised the state minimum wage from 4.25 USD to 5.05 USD while the minimum wage in Pennsylvania stays the same at 4.25.
* Data about the employment in the fast food restaurants (total number of employees in each restaurant) in NJ (0) and PA (1) were collected in February 1992 and in November 1992.
* Total 384 restaurants after removing null values.

Original paper: Card, D., & Krueger, A. B. (1993). Minimum wages and employment: A case study of the fast food industry in New Jersey and Pennsylvania. https://davidcard.berkeley.edu/papers/njmin-aer.pdf

## 1. Import libraries and data:

In [ ]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols

# Omiting WARNINGS
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Read data
df = pd.read_csv('employment.csv')

df.head()

In [ ]:
#
df.shape

In [ ]:
# Otra función que permite explorar datos:

df.info()

## 2. Descriptive Analysis

In [ ]:
# What is the mean of total number of employees in each restaurant by state?

df.groupby('state').mean()

In [ ]:
# Check by calculating the mean for each group directly
# 0 PA control group, 1 NJ treatment group

mean_emp_pa_before = df.groupby('state').mean().iloc[0, 0]
mean_emp_pa_after = df.groupby('state').mean().iloc[0, 1]
mean_emp_nj_before = df.groupby('state').mean().iloc[1, 0]
mean_emp_nj_after = df.groupby('state').mean().iloc[1, 1]

print(f'mean PA employment before: {mean_emp_pa_before:.2f}')
print(f'mean PA employment after: {mean_emp_pa_after:.2f}')
print(f'mean NJ employment before: {mean_emp_nj_before:.2f}')
print(f'mean NJ employment after: {mean_emp_nj_after:.2f}')

In [ ]:
# DiD
pa_diff = mean_emp_pa_after - mean_emp_pa_before
nj_diff = mean_emp_nj_after - mean_emp_nj_before
did = nj_diff - pa_diff

print(f'DID in mean employment is {did:.2f}')

## Implementing Regression

In [ ]:
# group D: 0 control group (PA), 1 treatment group (NJ)
# T: 0 before treatment (min wage raise), 1 after treatment
# DT: interaction of D * T

# data before the treatment
df_before = df[['total_emp_feb', 'state']]
df_before['T'] = 0
df_before.columns = ['total_emp', 'D', 'T'] # Rename columns

df_before

In [ ]:
# data after the treatment
df_after = df[['total_emp_nov', 'state']]
df_after['T'] = 1
df_after.columns = ['total_emp', 'D', 'T']

df_after

In [ ]:
# data for regression
df_reg = pd.concat( [df_before, df_after] )

# create the interaction 
df_reg['DT'] = df_reg["D"] * df_reg["T"]

df_reg

In [ ]:
# regression via 1

Y = df_reg["total_emp"]
X = df_reg[['D', 'T', 'DT']]
X = sm.add_constant(X)
est = sm.OLS(Y,X).fit()
print(est.summary())

In [ ]:
# regression via 2

ols = ols('total_emp ~ D + T + DT', data = df_reg).fit()
print(ols.summary())